<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/darvasboxforecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener backtesting openpyxl
# Gerekli kütüphaneler
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import openpyxl

# Uyarıları yok sayma
warnings.simplefilter(action='ignore', category=FutureWarning)

# Darvas Box stratejisini uygular
def darvas_box_strategy(data, bars_20=20, bars_50=50):
    df = data.copy()
    df['High_Max_20'] = df['High'].rolling(window=bars_20).max()
    df['Low_Min_20'] = df['Low'].rolling(window=bars_20).min()
    df['Darvas_Box_Up_20'] = df['Close'] > df['High_Max_20'].shift(1)
    df['Darvas_Box_Down_20'] = df['Close'] < df['Low_Min_20'].shift(1)
    df['High_Max_50'] = df['High'].rolling(window=bars_50).max()
    df['Low_Min_50'] = df['Low'].rolling(window=bars_50).min()
    df['Darvas_Box_Up_50'] = df['Close'] > df['High_Max_50'].shift(1)
    df['Darvas_Box_Down_50'] = df['Close'] < df['Low_Min_50'].shift(1)
    return df

# Kazanma oranını hesaplama fonksiyonu
def calculate_win_rate(df, buy_column, sell_column):
    buy_signals = df[df[buy_column] == True]
    sell_signals = df[df[sell_column] == True]

    win_count = 0
    total_count = min(len(buy_signals), len(sell_signals))

    for i in range(total_count):
        buy_price = buy_signals['Close'].iloc[i]
        sell_price = sell_signals['Close'].iloc[i]
        if sell_price > buy_price:
            win_count += 1

    if total_count > 0:
        return (win_count / total_count) * 100
    return 0

# TradingView API'ye bağlan
tv = TvDatafeed()

# BIST hisse senetlerini al
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Zaman dilimlerini tanımla
intervals = {
    '1': ('15 Dakika', Interval.in_15_minute),
    '2': ('30 Dakika', Interval.in_30_minute),
    '3': ('45 Dakika', Interval.in_45_minute),
    '4': ('1 Saat', Interval.in_1_hour),
    '5': ('2 Saat', Interval.in_2_hour),
    '6': ('4 Saat', Interval.in_4_hour),
    '7': ('Günlük', Interval.in_daily),
    '8': ('Haftalık', Interval.in_weekly),
    '9': ('Aylık', Interval.in_monthly)
}

# Kullanıcıdan zaman dilimi seçimini al
print("Lütfen bir zaman dilimi seçin:")
for key, (label, _) in intervals.items():
    print(f"{key}: {label}")
selected_interval = input("Seçiminizi yapın (1-9): ")

# Geçerli seçim kontrolü
if selected_interval in intervals:
    interval_name, interval_value = intervals[selected_interval]
    print(f"Seçilen zaman dilimi: {interval_name}")

    # Sonuçları saklamak için DataFrame'ler
    df_results = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Kazanma Oranı 20', 'Kazanma Oranı 50', 'Giriş Sinyali 20', 'Giriş Sinyali 50'])
    df_true_signals = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Kazanma Oranı 20', 'Kazanma Oranı 50', 'Giriş Sinyali 20', 'Giriş Sinyali 50'])

    # Hisseler üzerinde tarama
    for hisse in Hisseler:
        try:
            # Veri çek
            data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval_value, n_bars=500)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()

            # Darvas stratejisini uygula
            darvas_data = darvas_box_strategy(data)

            # Son fiyat ve sinyaller
            last_price = data['Close'].iloc[-1]
            buy_signal_20 = darvas_data['Darvas_Box_Up_20'].iloc[-1]
            buy_signal_50 = darvas_data['Darvas_Box_Up_50'].iloc[-1]

            # Kazanma oranlarını hesapla
            win_rate_20 = calculate_win_rate(darvas_data, 'Darvas_Box_Up_20', 'Darvas_Box_Down_20')
            win_rate_50 = calculate_win_rate(darvas_data, 'Darvas_Box_Up_50', 'Darvas_Box_Down_50')

            # Sonuçları ekle
            df_results.loc[len(df_results)] = [hisse, last_price, win_rate_20, win_rate_50, buy_signal_20, buy_signal_50]

            # True sinyalleri kontrol ve ekleme
            if buy_signal_20 or buy_signal_50:
                df_true_signals.loc[len(df_true_signals)] = [hisse, last_price, win_rate_20, win_rate_50, buy_signal_20, buy_signal_50]
                print(f"{hisse}, Son Fiyat: {last_price}, Kazanma Oranı 20: {win_rate_20:.2f}%, Kazanma Oranı 50: {win_rate_50:.2f}%, Giriş 20: {buy_signal_20}, Giriş 50: {buy_signal_50}")

        except Exception as e:
            print(f"Hata: {hisse} - {e}")

    # Sonuçları Excel'e kaydet
    excel_file = 'darvas_box_results.xlsx'
    with pd.ExcelWriter(excel_file) as writer:
        df_results.to_excel(writer, sheet_name='Tüm Sonuçlar', index=False)
        df_true_signals.to_excel(writer, sheet_name='True Sinyaller', index=False)

    print(f"Sonuçlar '{excel_file}' dosyasına kaydedildi.")
else:
    print("Geçersiz seçim! Lütfen 1 ile 9 arasında bir değer girin.")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-_1aso3m5
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-_1aso3m5
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=d5e7fa5d3879a41e105284220d52fc0b1f6c37469642add7408438e6f7fa5011
  Stored in directory: /tmp/pip-ephem-wheel-cache-xcaedstp/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=8ef9ddff42c8abababf8f40c08c4ee4e42a47cf4203acc3957f79df927d8306c
  Stored i

Lütfen bir zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 7
Seçilen zaman dilimi: Günlük
AKGRT, Son Fiyat: 6.7, Kazanma Oranı 20: 55.17%, Kazanma Oranı 50: 78.57%, Giriş 20: True, Giriş 50: True
ALGYO, Son Fiyat: 20.86, Kazanma Oranı 20: 22.22%, Kazanma Oranı 50: 37.50%, Giriş 20: True, Giriş 50: False
ALVES, Son Fiyat: 35.86, Kazanma Oranı 20: 60.00%, Kazanma Oranı 50: 0.00%, Giriş 20: True, Giriş 50: False
BIMAS, Son Fiyat: 505.0, Kazanma Oranı 20: 66.67%, Kazanma Oranı 50: 77.78%, Giriş 20: True, Giriş 50: False
BINBN, Son Fiyat: 114.6, Kazanma Oranı 20: 0.00%, Kazanma Oranı 50: 0.00%, Giriş 20: True, Giriş 50: False
BMSCH, Son Fiyat: 42.34, Kazanma Oranı 20: 0.00%, Kazanma Oranı 50: 0.00%, Giriş 20: True, Giriş 50: True
CMENT, Son Fiyat: 435.0, Kazanma Oranı 20: 75.56%, Kazanma Oranı 50: 76.19%, Giriş 20: True, Giriş 50: True
DAPGM, Son Fiyat: 6.08, Kazanma Oranı 20: 11.76%, Kazanma Ora